### Dataset Anexos Mágicos

In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
#Levanto tabla de ramos

ramos_df=pd.read_excel(r'C:\Users\eangenello\Desktop\PYTHON\input\ART\Dataset_AM\tabla_ramos.xlsx', index_col = 'cod_subramo')

In [3]:
ramos_df.head(2)

,cod_ramo,des_ramo,des_subramo
cod_subramo,,,
1.010.99,1.010.00,Incendio,Incendio - Incendio
1.020.01,1.020.00,Combinado Familiar e Integral,Comb. Fam. e Int. - Comb. Fam.


ANEXO 12

In [4]:
# Anexo 12 Primas - el primer trimestre tiene que ser 3

anexo12_df=pd.read_excel(r'C:\Users\eangenello\Desktop\PYTHON\input\ART\Dataset_AM\ANEXO_12.xlsx')

dataset_missing=pd.read_excel(r'C:\Users\eangenello\Desktop\PYTHON\input\ART\Dataset_AM\Missing_AM.xlsx', sheet_name='ANEXO_12')

anexo12_df=anexo12_df.append(dataset_missing)

anexo12_df.cod_cia=anexo12_df.cod_cia.astype('O')

anexo12_df=anexo12_df[(anexo12_df.cod_subramo=='1.050.01')|(anexo12_df.cod_subramo=='1.050.99')]

In [5]:
anexo12_df.head(1)

,cod_cia,des_cia,año,cod_subramo,Primas_Emitidas,Primas_Cedidas_Reaseguro,Compromisos_Tecnicos_del_Ej_Total,Compromisos_Tecnicos_del_Ej_AC_Reaseg,Compromisos_Tecnicos_del_Ej_AC_Asegurad,Compromisos_Tecnicos_del_Ej_Ant_Total,Compromisos_Tecnicos_del_Ej_Ant_AC_Reaseg,Compromisos_Tecnicos_del_Ej_Ant_AC_Asegurad,Primas_Devengadas_SD,Primas_Reaseguros_Activos,Primas_Cedidas_a_Retrocesion,Compromisos_Tecnicos_del_Ej,Compromisos_Tecnicos_del_Ej_Ant,Primas_Devengadas_RA,Primas_Devengadas_Total
193,32,BOSTON COMPAÑÍA ARGENTINA DE SEGUROS SOCIEDAD ...,2008-3,1.050.01,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [6]:
# Crea la columna Ramo y la completa buscando la información de la tabla de ramos

anexo12_ramo=anexo12_df

anexo12_ramo['ramo']=[ramos_df.loc[anexo12_df.cod_subramo[i],'des_ramo'] for i in anexo12_df.index]

In [7]:
# Crea una tabla del tipo "tabla dinámica de excel"
# Agrupa filas: por ID y ramo, columnas: año (año=aaaa-t) y suma las primas emitidas

anexo12_pivot=anexo12_ramo.pivot_table(index=['cod_cia','ramo'], values=['Primas_Emitidas'], columns=['año'], aggfunc=np.sum)

In [8]:
# Reemplaza los missing por nulos

anexo12_pivot=anexo12_pivot.replace(to_replace=np.nan, value=0)

In [9]:
#Creación base trimestral desacumulada

# Creo el esqueleto donde después va a incluir la información con un bucle. Lo que le estoy pidiendo es que utilice como esqueleto
# la misma estructura que tengo hasta ahora en 'anexo12_pivot' y que le modifique el nombre de las columnas a partir de 'multi_index'


multi_index=pd.MultiIndex(levels=[list(anexo12_pivot.columns.levels[0]), list(anexo12_pivot.columns.levels[1])],
                                    labels=list(anexo12_pivot.columns.labels),
                                    names=['variable', 'año'])


anexo12_trim=pd.DataFrame(index=anexo12_pivot.index , columns=multi_index)


In [10]:
# Defino la función que voy a utilizar para hacer el cálculo

# La función indica que si el período de análidis es 3, entonces que retorne el mismo monto de prima emitida. 
# Sino que al valor del período actual le reste el período anterior.

def trim(row,col,año):
    if anexo12_trim.loc[anexo12_trim.iloc[row].name[0]][col][año].name.split('-')[1]=='3':
        
        return anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][año]
    
    else:
        
        return anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][año]-anexo12_pivot.loc[anexo12_trim.iloc[row].name[0],anexo12_trim.iloc[row].name[1]][col][anexo12_trim.columns.levels[1][list(anexo12_trim.columns.levels[1]).index(año)-1]]
            

In [11]:
# Defino el bucle for al que voy a aplicar la función

variables=['Primas_Emitidas']

for j in range(len(anexo12_trim.index)):
    for col in variables:
        for año in list(anexo12_trim.columns.levels[1]):
            anexo12_trim.iloc[j][col][año]=trim(j,col,año)

In [12]:
anexo12_trim['cod_cia']=[anexo12_trim.iloc[i].name[0] for i in range(len(anexo12_trim.index))]
anexo12_trim['ramo']=[anexo12_trim.iloc[i].name[1] for i in range(len(anexo12_trim.index))]

In [13]:
# Obtengo el dataset con Ramo, Año y Prima Emitida Desagragada

anexo12_trim_melt=anexo12_trim.melt(id_vars=['cod_cia','ramo'])

anexo12_trim_melt=anexo12_trim_melt.drop({'variable','ramo'}, 1)

anexo12_trim_melt=anexo12_trim_melt.rename(columns = {'value':'Primas_Emitidas_Desagregadas'})


In [14]:
# Agrupo el Anexo 12 por año y compañía

anexo12_df_agrup=anexo12_df.groupby(['cod_cia','año'], as_index=False)["Primas_Emitidas","Primas_Devengadas_Total"].sum()

In [15]:
# Junto ambos datasets: Prima emitida desagregada con Prima Emitida

Dataset_AM=anexo12_trim_melt.merge(anexo12_df_agrup, how='outer', on=['cod_cia','año'])

ANEXO 8

In [16]:
anexo8_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\ANEXO_8.xlsx', usecols= "A:L,N,O,AB")

dataset_missing=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Missing_AM.xlsx', sheet_name='ANEXO_8')

anexo8_df=anexo8_df.append(dataset_missing)

anexo8_df=anexo8_df[(anexo8_df.cod_subramo=='1.050.01')|(anexo8_df.cod_subramo=='1.050.99')]

anexo8_df_agrup=anexo8_df.groupby(['cod_cia','año'], as_index=False)['RSP_Administrativos','RSP_Juicios','RSP_Mediaciones','IBNR','IBNER','Acreed_por_Premios_a_Devol','ILT_a_Pagar','Prest_en_Espec_a_pagar','Siniestros_Liquidados_a_Pagar','Siniestros_en_Proceso_de_Liquidacion','Deudas_con_Asegurados'].sum()

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [17]:
# Junto datasets: Anexo 12 + Anexo 8

Dataset_AM=Dataset_AM.merge(anexo8_df_agrup, how='outer', on=['cod_cia','año'])

ANEXO 11

In [18]:
anexo11_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\ANEXO_11.xlsx', usecols='A:D,J,K,AD')

dataset_missing=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Missing_AM.xlsx', sheet_name='ANEXO_11')

anexo11_df=anexo11_df.append(dataset_missing)

anexo11_df=anexo11_df[(anexo11_df.cod_subramo=='1.050.01')|(anexo11_df.cod_subramo=='1.050.99')]

anexo11_df_agrup=anexo11_df.groupby(['cod_cia','año'], as_index=False)['Conting_y_Desvios_de_Siniestralidad_ART','Por_Resultado_Negativo_ART','Compromisos_Tecnicos'].sum()

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [19]:
# Junto datasets: Anexo 12 + Anexo 8 + Anexo 11

Dataset_AM=Dataset_AM.merge(anexo11_df_agrup, how='outer', on=['cod_cia','año'])

ANEXO 14

In [20]:
# Importo y preparo Anexo 14

anexo14_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\ANEXO_14.xlsx', usecols='A:E,G,N')

dataset_missing=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Missing_AM.xlsx', sheet_name='ANEXO_14')

anexo14_df=anexo14_df.append(dataset_missing)

anexo14_df=anexo14_df[(anexo14_df.cod_subramo=='1.050.01')|(anexo14_df.cod_subramo=='1.050.99')]

anexo14_df_agrup=anexo14_df.groupby(['cod_cia','año'], as_index=False)['Comisiones','Sub_Total_GP','Sub_total_GE'].sum()

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [21]:
# Junto datasets: Anexo 12 + Anexo 8 + Anexo 11 + Anexo 14

Dataset_AM=Dataset_AM.merge(anexo14_df_agrup, how='outer', on=['cod_cia','año'])

BALANCE_2

In [22]:
# Importo y preparo Siniestros en Juicio y Pendiente de Liquidación de BALANCE_2

balances2_df=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\BALANCES_2.xlsx', usecols='A:F')
dataset_missing=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Missing_AM.xlsx', sheet_name='BALANCES_2')
balances2_df=balances2_df.append(dataset_missing)

#Filtro subramo y codigo de cuenta
balances2_df=balances2_df[(balances2_df.cod_subramo=='1.050.01')|(balances2_df.cod_subramo=='1.050.99')]
balances2A_df=balances2_df[balances2_df.cod_cuenta=='2.01.01.01.01.18.01.00']
balances2B_df=balances2_df[balances2_df.cod_cuenta=='2.01.01.01.01.18.02.00']

# Siniestros ART - En mediación
balances2A_df['Siniestros ART - En mediación']=balances2A_df['importe']
balances2A_df=balances2A_df.drop(['cod_cuenta', 'importe'],1)

# Siniestros ART - En juicio
balances2B_df['Siniestros ART - En juicio']=balances2B_df['importe']
balances2B_df=balances2B_df.drop(['cod_cuenta', 'importe'],1)

# Junto ambas variables y agrupo
BALANCES_2_df=balances2A_df.merge(balances2B_df, how='outer', on=['cod_cia','des_cia','año','cod_subramo'])
BALANCES_2_df_agrup=BALANCES_2_df.groupby(['cod_cia','año'], as_index=False)['Siniestros ART - En mediación','Siniestros ART - En juicio'].sum()

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  del sys.path[0]
C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [23]:
# Junto datasets: Anexo 12 + Anexo 8 + Anexo 11 + Anexo 14 + BALANCES_2

Dataset_AM=Dataset_AM.merge(BALANCES_2_df_agrup, how='outer', on=['cod_cia','año'])

In [24]:
#Filtro por 2012 en adelante y renombro la columna 'año' por 'periodo'

Dataset_AM['year'] = pd.DatetimeIndex(Dataset_AM['año']).year
Dataset_AM= Dataset_AM.drop(Dataset_AM[Dataset_AM.year < 2012].index)


In [25]:
    # Creo variables para contruir 'Año-Mes'

Dataset_AM['quarter'] = pd.DatetimeIndex(Dataset_AM['año']).month

Dataset_AM['month']=Dataset_AM['quarter']

In [26]:
# Redefino month para que quede acorde al último mes del trimestre correspondiente

for i in Dataset_AM.index:
    if Dataset_AM.quarter[i]==1:
        Dataset_AM.month[i]=3
    elif Dataset_AM.quarter[i]==2:
        Dataset_AM.month[i]=6
    elif Dataset_AM.quarter[i]==3:
        Dataset_AM.month[i]=9
    else:
        Dataset_AM.month[i]=12

C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
C:\Users\eangenello\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
C:\Users\eangenello

In [27]:
Dataset_AM['Periodo'] = Dataset_AM.year.astype(str).str.cat(Dataset_AM.month.astype(str), sep='-')

Dataset_AM=Dataset_AM.drop(['year','quarter','month'],1)

In [28]:
Dataset_AM['Periodo_balance']=Dataset_AM['año']
Dataset_AM=Dataset_AM.drop('año',1)

In [29]:
#Creo variable prima emitida negativa

Dataset_AM['pe_negativa']=[1 if Dataset_AM.Primas_Emitidas.iloc[i]<0 else 0 for i in range(len(Dataset_AM.index))]

Saldo a amortizar dif. reserva pto. 33.4.1.6 (-) y Reserva Especial ART Res. 41.155

In [30]:
# Importo base construida a partir del SINENSUP
Dataset_Sinensup=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Dataset_NEECC_Sinensup.xlsx', sheet_name = 'Sinensup')

Dataset_Sinensup=Dataset_Sinensup.drop('des_cia',1)

In [31]:
# Junto datasets: Anexo 12 + Anexo 8 + Anexo 11 + Anexo 14 + BALANCES2 + Sinensup

Dataset_AM=Dataset_AM.merge(Dataset_Sinensup, how='left', on=['cod_cia','Periodo_balance'])

NOTAS EECC

In [32]:
# Importo base construida a partir de las Notas a los Estados Contables
# [Reserva por Pasivo Global], [Cantidad de Juicios Abiertos con Pagos Parciales] y [Total de Pagos de los Casos Abiertos con Juicios]

Dataset_NEECC=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_AM\Dataset_NEECC_Sinensup.xlsx', sheet_name = 'NEECC', usecols='A:H')

Dataset_NEECC=Dataset_NEECC.drop('des_cia',1)

In [33]:
# Junto datasets: Anexo 12 + Anexo 8 + Anexo 11 + Anexo 14 + BALANCES2 + Sinensup + NEECC

Dataset_AM=Dataset_AM.merge(Dataset_NEECC, how='left', on=['cod_cia','Periodo_balance'])


Dataset SRT (Masa Salarial y Trabajadores Cubiertos)

In [34]:
Dt_SRT=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dt_SART.xlsx', usecols='A:G,J' )


In [35]:
# Armo el dataset de Trabajadores Cubiertos: De esta variable sólo necesito la foto al último mes del trimestre

# Elimino Masa Salarial

Dt_SRT_TC=Dt_SRT.drop('Masa_Salarial', 1)

# Elimino los meses que no corresponden al último mes de cada trimestre

Dt_SRT_TC['month'] = pd.DatetimeIndex(Dt_SRT_TC['Periodo']).month

Dt_SRT_TC = Dt_SRT_TC.drop(Dt_SRT_TC[Dt_SRT_TC.month==1].index,0)
Dt_SRT_TC = Dt_SRT_TC.drop(Dt_SRT_TC[Dt_SRT_TC.month==2].index,0)
Dt_SRT_TC = Dt_SRT_TC.drop(Dt_SRT_TC[Dt_SRT_TC.month==4].index,0)
Dt_SRT_TC = Dt_SRT_TC.drop(Dt_SRT_TC[Dt_SRT_TC.month==5].index,0)
Dt_SRT_TC = Dt_SRT_TC.drop(Dt_SRT_TC[Dt_SRT_TC.month==7].index,0)
Dt_SRT_TC = Dt_SRT_TC.drop(Dt_SRT_TC[Dt_SRT_TC.month==8].index,0)
Dt_SRT_TC = Dt_SRT_TC.drop(Dt_SRT_TC[Dt_SRT_TC.month==10].index,0)
Dt_SRT_TC = Dt_SRT_TC.drop(Dt_SRT_TC[Dt_SRT_TC.month==11].index,0)

Dt_SRT_TC = Dt_SRT_TC.drop('month',1)
Dt_SRT_TC = Dt_SRT_TC.drop('denominacion_corta',1)

In [36]:
DF_exportar= Dataset_AM.merge(Dt_SRT_TC, how='outer', on=['cod_cia','Periodo'])

In [37]:
# Armo el dataset de Trabajadores Cubiertos: De esta variable necesito el acumulado trimestral para poder hacer la comparación
# con la prima emitida trimestral

# Elimino Trabajadores Cubiertos

Dt_SRT_MS=Dt_SRT.drop('Trabajadores_Cubiertos', 1)

# Creo las variables mes y año para construir la variable Periodo Balance

Dt_SRT_MS['year'] = pd.DatetimeIndex(Dt_SRT_MS['Fecha']).year
Dt_SRT_MS['month'] = pd.DatetimeIndex(Dt_SRT_MS['Fecha']).month

In [38]:
# Defino la función que va a asignar el trimestre a partir de la variable mes

def trim(row):
    if Dt_SRT_MS.month.iloc[row]==1 or Dt_SRT_MS.month.iloc[row]==2 or Dt_SRT_MS.month.iloc[row]==3:
        return 1
    
    elif Dt_SRT_MS.month.iloc[row]==4 or Dt_SRT_MS.month.iloc[row]==5 or Dt_SRT_MS.month.iloc[row]==6:
        return 2
    
    elif Dt_SRT_MS.month.iloc[row]==7 or Dt_SRT_MS.month.iloc[row]==8 or Dt_SRT_MS.month.iloc[row]==9:
        return 3
    
    else:
        return 4

In [39]:
# Creo la variable Trimestre y le aplico la función definida

Dt_SRT_MS['trimestre']=[trim(i) for i in range(len(Dt_SRT_MS.index))]

In [40]:
# Creo la variable Periodo Balance a partir de las varibles year y trimestre

Dt_SRT_MS['Periodo_balance'] = Dt_SRT_MS.year.astype(str).str.cat(Dt_SRT_MS.trimestre.astype(str), sep='-')

In [41]:
# Agrupo en función de las variables cod_cia y Periodo Balance

Dt_SRT_MS_agrup=Dt_SRT_MS.groupby(['cod_cia','Periodo_balance'], as_index=False)['Masa_Salarial'].sum()

Dt_SRT_MS_agrup=Dt_SRT_MS_agrup[Dt_SRT_MS_agrup.cod_cia!=9999]

# A LO NICO!
# Masa_Salarial_Trim=Dt_SRT_MS.pivot_table(index=['cod_cia','year','trimestre'], values='Masa_Salarial', aggfunc=np.sum)
# Masa_Salarial_Trim['entidad']=[Masa_Salarial_Trim.iloc[i].name[0] for i in range(len(Masa_Salarial_Trim.index))]
# Masa_Salarial_Trim['año']=[Masa_Salarial_Trim.iloc[i].name[1] for i in range(len(Masa_Salarial_Trim.index))]
# Masa_Salarial_Trim['trim']=[Masa_Salarial_Trim.iloc[i].name[2] for i in range(len(Masa_Salarial_Trim.index))]

In [42]:
DF_exportar= DF_exportar.merge(Dt_SRT_MS_agrup, how='outer', on=['cod_cia','Periodo_balance'])

DENOMINACIÓN CORTA COMPAÑÍAS

In [43]:
# Importo nombre compañías

Dataset_compañías=pd.read_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\input\ART\Dataset_Compañías.xlsx', sheet_name='AM')

In [44]:
# Junto datasets: Anexo 2 + Anexo 8 + Anexo 11 + Anexo 14 + BALANCES_2 + Sinensup + NEEC + Compañías

DF_exportar=DF_exportar.merge(Dataset_compañías, how='left', on='cod_cia')

COMPAÑÍAS ACTIVAS

In [45]:
#Creo variable compañías activas y filtro

cias=[64,79,254,259,261,348,412,440,605,616,618,619,620,621,624,626,726,862,890,9999]
DF_exportar['cias_activas']=[1 if np.isin(DF_exportar.cod_cia.iloc[i],cias)==True else 0 for i in range(len(DF_exportar.index))]

DF_exportar=DF_exportar[(DF_exportar.cias_activas==1)]

Reordeno 

In [46]:
# Listo las etiquetas de las columnas

cols = list(DF_exportar.columns.values)
cols

['cod_cia',
 'Primas_Emitidas_Desagregadas',
 'Primas_Emitidas',
 'Primas_Devengadas_Total',
 'RSP_Administrativos',
 'RSP_Juicios',
 'RSP_Mediaciones',
 'IBNR',
 'IBNER',
 'Acreed_por_Premios_a_Devol',
 'ILT_a_Pagar',
 'Prest_en_Espec_a_pagar',
 'Siniestros_Liquidados_a_Pagar',
 'Siniestros_en_Proceso_de_Liquidacion',
 'Deudas_con_Asegurados',
 'Conting_y_Desvios_de_Siniestralidad_ART',
 'Por_Resultado_Negativo_ART',
 'Compromisos_Tecnicos',
 'Comisiones',
 'Sub_Total_GP',
 'Sub_total_GE',
 'Siniestros ART - En mediación',
 'Siniestros ART - En juicio',
 'Periodo',
 'Periodo_balance',
 'pe_negativa',
 'Saldo a amortizar dif. reserva pto. 33.4.1.6 (-)',
 'Reserva Especial ART Res. 41.155',
 'Cantidad de Juicios',
 'Reserva por Pasivo Global',
 'Cantidad de Juicios Abiertos con Pagos Parciales',
 'Total de Pagos de los Casos Abiertos con Juicios',
 'Valuación de reservas por reclamaciones judiciales',
 'Fecha',
 'Trabajadores_Cubiertos',
 'Empleadores_Asegurados',
 'Cuota_Pactada_Porcen

In [47]:
# Reordeno las columnas

reord= ['cod_cia',
 'denominacion_corta',
 'des_cia',
 'Periodo',
 'Periodo_balance',
 'Fecha',
 'Primas_Emitidas_Desagregadas',
 'Primas_Emitidas',
 'Primas_Devengadas_Total',
 'RSP_Administrativos',
 'RSP_Juicios',
 'RSP_Mediaciones',
 'IBNR',
 'IBNER',
 'Acreed_por_Premios_a_Devol',
 'ILT_a_Pagar',
 'Prest_en_Espec_a_pagar',
 'Siniestros_Liquidados_a_Pagar',
 'Siniestros_en_Proceso_de_Liquidacion',
 'Deudas_con_Asegurados',
 'Conting_y_Desvios_de_Siniestralidad_ART',
 'Por_Resultado_Negativo_ART',
 'Compromisos_Tecnicos',
 'Comisiones',
 'Sub_Total_GP',
 'Sub_total_GE',
 'Siniestros ART - En mediación',
 'Siniestros ART - En juicio',
 'pe_negativa',
 'Saldo a amortizar dif. reserva pto. 33.4.1.6 (-)',
 'Reserva Especial ART Res. 41.155',
 'Cantidad de Juicios',
 'Reserva por Pasivo Global',
 'Cantidad de Juicios Abiertos con Pagos Parciales',
 'Total de Pagos de los Casos Abiertos con Juicios',
 'Valuación de reservas por reclamaciones judiciales',
 'Trabajadores_Cubiertos',
 'Empleadores_Asegurados',
 'Cuota_Pactada_Porcentaje',
 'Masa_Salarial',
 'cias_activas']
    
DF_exportar=DF_exportar[reord]

In [48]:
# Reemplazo missing por cero

DF_exportar=DF_exportar.replace(to_replace=np.nan, value=0)

EXPORTO

In [49]:
DF_exportar.to_excel(r'C:\Users\eangenello\Desktop\TECNICA\Python\output\Dt_Anexos_Magicos.xlsx', sheet_name='Dt_Anexos_Magicos', index=False)